# PhageScope data exploration

Basic packages 

The PhageScope data are downloaded via snakemake and reports generated on up to 50k randomly selected datapoint. 


## File list



In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

data_dir = "../data"


### Protein Metadata

Temporary creation of big lists for later use. 

In [ ]:
# Accumulators
chunk_num = 0
total_rows = 0
phage_ids = []
protein_ids = []
source = [] # Source of the protein, e.g. DDBJ, CHVD, etc. match the directory names for the fasta files


for chunk in pd.read_csv(os.path.join(data_dir, "merged/merged_annotated_proteins_metadata.csv"), chunksize=10000):
    total_rows += len(chunk)
    phage_ids.extend(chunk['Phage_ID'].tolist())
    protein_ids.extend(chunk['Protein_ID'].tolist())
    source.extend(chunk['Phage_source'].tolist())


    # ------------------------------
    chunk_num += 1
    if chunk_num >= 55:  # Limit to 5 chunks for demonstration
        break

In [ ]:
print(pd.Series(source).unique())

In [ ]:

#transcription_terminator_metadata = pd.read_csv(os.path.join(data_dir, "merged/merged_transcription_terminator_metadata.csv"))
#phage_metadata = pd.read_csv(os.path.join(data_dir, "merged/merged_phage_metadata.csv"))
#phage_protein_metadata = pd.read_csv(os.path.join(data_dir, "merged/merged_phage_protein_metadata.csv"))

## Protein Fasta

Some sources have one fasta file per protein, other have one fasta file containing all proteins. 

In total, there are more than 23 milion fasta files (just protein !) Some sources have one file containing multiple sequences, while other sources have one fasta file per sequence. 

```
Unique sources: 13
RefSeq: 480642 FASTA files
IGVD: 1 FASTA files
PhagesDB: 351898 FASTA files
GPD: 7616044 FASTA files
GOV2: 1 FASTA files
CHVD: 1 FASTA files
MGV: 10517011 FASTA files
DDBJ: 17391 FASTA files
STV: 1 FASTA files
EMBL: 11095 FASTA files
TemPhD: 3465586 FASTA files
GVD: 746146 FASTA files
Genbank: 217128 FASTA files
Total fasta files found: 23422945
```

To obtain the sequence from the protein ID, we should be able to query the sequences at will and for this, having a coeherent database is vital. We will then merge the sequences in one file per source which should allow later to create a performant database, which is not possible with 23 million files. 

The structure is like this: 

```
protein_fasta/
  └── source1/
        ├── source1/
        │     ├── phageA/
        │     │     ├── file1.fasta
        │     │     └── file2.fasta
        │     └── phageB/
        │           └── file3.fasta
  └── source2/
        └── single.fasta

```

### Merging Fasta files per source

Using the script `merge_protein_fasta.py` with the snakemake rules `snakemake --cores 2 data/protein_fasta_merged/DDBJ.fasta` for each file, it will merge the files if necessary into data/protein_fasta_merged. 

Visualize with `snakemake --cores 4 --dag | dot -Tsvg > dag_all.svg`



In [ ]:
fasta_files